# Steps

This document is research of semantic similarity between texts implementins scripts steps.

In [1]:
# import used modules
import os
import numpy as np
import time
import pickle

In [2]:
# import a model and a module for semantic analys
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
# import our module for semantic analys
import sys
sys.path.append('../../')
import sem_analysis as sa


Loading the model...
Processing input...


## 1. Extract Sentences From Documents And Their Preprocessing

In [4]:
files = dict()
for i in os.listdir():
    if os.path.isdir(i) and i.isdigit():
        for j in os.listdir(i):
            name = i + '/' + j
            f = open(name, 'r')
            files[i + '/' + j] = f.read()
            f.close()

In [5]:
import numpy as np
n = len(files.keys())
D = np.zeros((n, n))
keys = list(files.keys())
keys.sort()
texts = [files[i] for i in keys]
len(texts)

71

Now we transform our phrases to format for [RusVectores](https://rusvectores.org/ru/). See [tutorial for RusVectores](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb).

In [6]:
name_pickle = 'texts_transform.pickle'
if not name_pickle in os.listdir():
    t = time.time()
    texts_transform = list()
    for i in texts:
        texts_transform.append(sa.tag_ud(i))
        if len(texts_transform) % 10 == 0:
            print(len(texts_transform), time.time() - t)
    with open(name_pickle, 'wb') as f:
        pickle.dump(texts_transform, f)
        f.close()
else:
    with open(name_pickle, 'rb') as f: 
        texts_transform = pickle.load(f)
        f.close()

## 2. Quallity Of Separation

In [7]:
name_pickle = 'texts_transform.pickle'
with open(name_pickle, 'rb') as f: 
    texts_transform = pickle.load(f)
    f.close()
centers = list()
for j in texts_transform:
    new = [model[i] for i in j if model.__contains__(i)]
    centers.append(sum(new)/len(new))

In [8]:
names = list(files.keys())
names.sort()
list_ = [str(i) for i in range(1, 14)]
_ = [i.split('/')[0] for i in names]
cur_dict=dict()
for ind, i in enumerate(_):
    if not cur_dict.__contains__(i):
        cur_dict[i] = list()
    cur_dict[i].append(centers[ind])

In [9]:
spheres = dict()
for i in cur_dict:
    c = sum(cur_dict[i]) / len(cur_dict[i])
    spheres[i] = (c, max([np.linalg.norm(c - j) for j in cur_dict[i]]))

In [10]:
print([(i,spheres[i][1]) for i in spheres])
full_spheres = {i: list() for i in spheres}
for ind, j in enumerate(centers):
    m, r = None, np.inf
    for key in spheres:
        if m is None or r > np.linalg.norm(spheres[key][0] - j):
            m, r = key, np.linalg.norm(spheres[key][0] - j)
    full_spheres[m].append(ind)

[('4', 0.21799523), ('11', 0.21251673), ('10', 0.16417822), ('1', 0.2517743), ('6', 0.3560091), ('5', 0.19442385), ('12', 0.0), ('2', 0.20956105), ('7', 0.28540248), ('9', 0.20796265), ('3', 0.19966942), ('8', 0.22441144), ('0', 0.17428154)]


In [11]:
true_spheres = {i:[ind for ind,j in enumerate(names) if i ==j.split('/')[0]] for i in spheres}
def metric(list1, list2):
    TP = len([i for i in list2 if i in list1])
    FP = len([i for i in list2 if not i in list1])
    FN = len([i for i in list1 if not i in list2])
    return TP/(TP+FN), TP/(FP+TP)
def full_acc(true_spheres, full_spheres):
    s, fs = 0., 0.
    for i in true_spheres:
        s += len([j for j in full_spheres[i] if j in true_spheres[i]])
        fs += len(true_spheres[i])
    print('Accuracy', s/ fs)

keys = list(true_spheres.keys())
keys = [int(i) for i in keys]
keys.sort()
keys = [str(i) for i in keys]
print("Number, Recall, Precision")
for i in keys:
    print(i, metric(true_spheres[i], full_spheres[i]))
full_acc(true_spheres, full_spheres)

Number, Recall, Precision
0 (1.0, 1.0)
1 (0.75, 1.0)
2 (0.75, 1.0)
3 (1.0, 0.875)
4 (0.7777777777777778, 0.6363636363636364)
5 (0.5, 0.5)
6 (0.625, 0.8333333333333334)
7 (0.75, 0.75)
8 (0.8, 0.5714285714285714)
9 (0.8571428571428571, 1.0)
10 (1.0, 0.8)
11 (0.6666666666666666, 1.0)
12 (1.0, 1.0)
Accuracy 0.7887323943661971


## 3. Research Distance Between Steps

In [12]:
name_pickle = 'texts_transform.pickle'
with open(name_pickle, 'rb') as f: 
    texts_transform = pickle.load(f)
    f.close()
centers = list()
for j in texts_transform:
    new = [model[i] for i in j if model.__contains__(i)]
    centers.append(sum(new)/len(new))

In [13]:
names = list(files.keys())
names.sort()
_ = [i.split('/')[0] for i in names]
cur_dict=dict()
for ind, i in enumerate(_):
    if not cur_dict.__contains__(i):
        cur_dict[i] = list()
    cur_dict[i].append(centers[ind])

In [14]:
n = len(spheres.keys())
D_centers = np.zeros((n,n))
keys = [i for i in range(0, n)]
for i in keys:
    for j in keys:
        D_centers[i,j] = np.linalg.norm(spheres[str(i)][0] - spheres[str(j)][0])

### 3.1. Distance Between Steps Centers

In [16]:
import pandas as pd
pd.DataFrame(D_centers)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000000,0.194131,0.172467,0.168954,0.160711,0.198725,0.184992,0.162391,0.218876,0.254127,0.154296,0.183489,0.310002
1,0.194131,0.000000,0.191037,0.170202,0.181411,0.213168,0.168360,0.192671,0.211490,0.250965,0.151607,0.185810,0.300835
2,0.172467,0.191037,0.000000,0.154325,0.135318,0.155952,0.147221,0.139953,0.201676,0.241108,0.184066,0.162102,0.240007
3,0.168954,0.170202,0.154325,0.000000,0.128794,0.177108,0.109663,0.148081,0.188185,0.229726,0.126717,0.129200,0.296957
4,0.160711,0.181411,0.135318,0.128794,0.000000,0.136217,0.130027,0.121715,0.167401,0.221174,0.154763,0.129561,0.286475
5,0.198725,0.213168,0.155952,0.177108,0.136217,0.000000,0.156411,0.105978,0.219177,0.280072,0.209274,0.175100,0.299323
6,0.184992,0.168360,0.147221,0.109663,0.130027,0.156411,0.000000,0.149320,0.198547,0.245594,0.161364,0.126757,0.301409
7,0.162391,0.192671,0.139953,0.148081,0.121715,0.105978,0.149320,0.000000,0.181603,0.236673,0.172451,0.167455,0.273387
8,0.218876,0.211490,0.201676,0.188185,0.167401,0.219177,0.198547,0.181603,0.000000,0.100414,0.204678,0.215522,0.227614
9,0.254127,0.250965,0.241108,0.229726,0.221174,0.280072,0.245594,0.236673,0.100414,0.000000,0.244484,0.261868,0.229756


### 3.2. Statistic Information

In [17]:

D_st = np.array([(D_centers+1000*np.eye(n)).min(axis = 1), (D_centers+1000*np.eye(n)).argmin(axis = 1), 
                 D_centers.max(axis = 1), D_centers.argmax(axis = 1),
                 D_centers.mean(axis = 1)])
D_st = D_st.transpose()
pd.DataFrame(D_st, columns = ['MIN', 'ARGMIN','MAX', 'ARGMAX', 'MEAN'])

,MIN,ARGMIN,MAX,ARGMAX,MEAN
0,0.154296,10.0,0.310002,12.0,0.181782
1,0.151607,10.0,0.300835,12.0,0.185514
2,0.135318,4.0,0.241108,9.0,0.163479
3,0.109663,6.0,0.296957,12.0,0.155993
4,0.121715,7.0,0.286475,12.0,0.150274
5,0.105978,7.0,0.299323,12.0,0.178962
6,0.109663,3.0,0.301409,12.0,0.159974
7,0.105978,5.0,0.273387,12.0,0.157822
8,0.100414,9.0,0.227614,12.0,0.179629
9,0.100414,8.0,0.280072,5.0,0.215074


## 3.3 Internal Distance